<a href="https://colab.research.google.com/github/feiduobaby/good-first-issue/blob/main/Homework8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Data Preparation

In [ ]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
!unzip data.zip

In [4]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:
import torch.nn as nn
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # 1) Conv layer: in 3 ch, out 32 ch, kernel 3x3
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
        # 2) Max pooling: 2x2
        self.pool = nn.MaxPool2d(kernel_size=(2, 2))
        # After conv1 (no padding): input (3,200,200) -> (32, 198, 198)
        # After maxpool 2x2: -> (32, 99, 99)
        # Flatten size = 32 * 99 * 99
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()  # binary output

    def forward(self, x):
        # x: (batch, 3, 200, 200)
        x = self.conv1(x)          # (batch, 32, 198, 198)
        x = F.relu(x)
        x = self.pool(x)           # (batch, 32, 99, 99)
        x = torch.flatten(x, 1)    # (batch, 32*99*99)
        x = self.fc1(x)            # (batch, 64)
        x = F.relu(x)
        x = self.fc2(x)            # (batch, 1)
        x = self.sigmoid(x)        # probability for class 1
        return x


In [6]:
model = SimpleCNN()

optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

criterion = nn.MSELoss()  # expects outputs in (0,1) and target as float 0/1

In [7]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


In [16]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_dir = "/content/data"

transform = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),              # (3, 200, 200) tensor
])

train_dataset = datasets.ImageFolder(root=f"{data_dir}/train", transform=transform)
validation_dataset = datasets.ImageFolder(root=f"{data_dir}/test",   transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

print(train_dataset.classes)      # ['curly', 'straight']，顺序就是类别到数字的映射
print(train_dataset.class_to_idx) # 比如 {'curly': 0, 'straight': 1}


['curly', 'straight']
{'curly': 0, 'straight': 1}


In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [17]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.2275, Acc: 0.4875, Val Loss: 0.2245, Val Acc: 0.4876
Epoch 2/10, Loss: 0.2180, Acc: 0.4875, Val Loss: 0.2198, Val Acc: 0.4876
Epoch 3/10, Loss: 0.2077, Acc: 0.4875, Val Loss: 0.2189, Val Acc: 0.4876
Epoch 4/10, Loss: 0.1986, Acc: 0.4875, Val Loss: 0.2156, Val Acc: 0.4876
Epoch 5/10, Loss: 0.1991, Acc: 0.4875, Val Loss: 0.2255, Val Acc: 0.4876
Epoch 6/10, Loss: 0.1980, Acc: 0.4875, Val Loss: 0.2136, Val Acc: 0.4876
Epoch 7/10, Loss: 0.1946, Acc: 0.4875, Val Loss: 0.2113, Val Acc: 0.4876
Epoch 8/10, Loss: 0.1967, Acc: 0.4875, Val Loss: 0.2112, Val Acc: 0.4876
Epoch 9/10, Loss: 0.1878, Acc: 0.4875, Val Loss: 0.2145, Val Acc: 0.4876
Epoch 10/10, Loss: 0.1793, Acc: 0.4875, Val Loss: 0.2114, Val Acc: 0.4876
